<a href="https://colab.research.google.com/github/Anunineleaps6/Capstone_Sales/blob/main/Capstone_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gspread oauth2client pandas numpy


In [2]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("/content/drive/MyDrive/CAPSTONE_FINAL/creds.json", scope)
client = gspread.authorize(creds)


In [3]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta


In [4]:
!pip install phonenumbers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 38.8 MB/s eta 0:00:00


In [5]:
import phonenumbers

def generate_phone(country, region_mapping):
    # Try to get the country code from the region_mapping first
    code = None
    for region, data in region_mapping.items():
        if country in data["countries"]:
            code = data["dial_code"].get(country)
            break

    if code:
        try:
            # Generate a random 10-digit number
            local_number = ''.join([str(random.randint(0, 9)) for _ in range(10)])
            # Attempt to parse and format using phonenumbers
            # Note: This might not create *valid* numbers, but will follow the format
            # We need a country code without the '+' for the parse function
            country_code_for_parse = code.lstrip('+')
            # We need to find a region code (like US, GB, IN) for the parse function
            # This is a simplified approach; a more robust solution would map countries to region codes
            country_region_code = ""
            for region_key, data in region_mapping.items():
                if country in data.get("countries", []):
                     # This is a simplification; need a proper country to region code mapping
                     # For now, we can try to infer from common country codes or use a default
                     # A more accurate approach would involve a comprehensive country code to region code mapping
                     # Let's use a placeholder logic or rely on the dial code if a direct mapping is complex
                     pass # Will refine this

            # A better approach might be to directly format a random number with the known dial code
            # We'll stick to generating a random 10-digit number and prefixing the dial code
            # as phonenumbers.parse requires a known region or a leading '+' which we already have

            # Let's refine the phonenumbers usage or simplify if direct generation is sufficient
            # Given the user wants the format "+[country_code]-[10_digits]", direct generation is simpler
            number = ''.join([str(random.randint(0, 9)) for _ in range(10)])
            return f"{code}-{number}"

        except Exception as e:
            print(f"Error generating phone for {country} with code {code}: {e}")
            # Fallback to a generic format if phonenumbers fails
            return f"{code}-{random.randint(1000000000, 9999999999)}"
    else:
        # Fallback for countries not in region_mapping
        return f"+00-{random.randint(1000000000, 9999999999)}"

# Now, modify the function call in the lead generation loop in cell 9212e2f3
# (This modification will be done in the next modify_cells command)

In [6]:
#LEADS
import pandas as pd
import random
import uuid
from datetime import datetime, timedelta

# ------------------------------------------
# 1. Region, Country, Dial Code Mapping (Used for phone number generation)
# ------------------------------------------
region_mapping = {
    "NA": {"countries": ["USA", "Canada"], "dial_code": {"USA": "+1", "Canada": "+1"}, "industries": ["Technology", "Healthcare", "Finance", "Retail"], "services": ["Data", "Marketing", "Cloud"]},
    "EU": {"countries": ["UK", "Germany", "France", "Netherlands", "Switzerland", "Sweden", "Israel"], "dial_code": {"UK": "+44", "Germany": "+49", "France": "+33", "Netherlands": "+31", "Switzerland": "+41", "Sweden": "+46", "Israel": "+972"}, "industries": ["Automotive", "Pharmaceuticals", "Banking", "Consulting"], "services": ["Consulting", "Engineering", "Operations"]},
    "APAC": {"countries": ["India", "Japan", "Singapore", "China", "Indonesia", "Australia", "New Zealand", "S. Korea"], "dial_code": {"India": "+91", "Japan": "+81", "Singapore": "+65", "China": "+86", "Indonesia": "+62", "Australia": "+61", "New Zealand": "+64", "S. Korea": "+82"}, "industries": ["Technology", "E-Commerce", "Manufacturing", "Finance"], "services": ["AI", "Staffing", "T&M"]},
    "ME": {"countries": ["UAE", "Saudi Arabia", "Qatar", "Bahrain"], "dial_code": {"UAE": "+971", "Saudi Arabia": "+966", "Qatar": "+974", "Bahrain": "+973"}, "industries": ["Energy", "Real Estate", "Banking"], "services": ["Data", "Consulting"]},
    "LA": {"countries": ["Brazil", "Mexico", "Argentina", "Chile", "Colombia", "Peru"], "dial_code": {"Brazil": "+55", "Mexico": "+52", "Argentina": "+54", "Chile": "+56", "Colombia": "+57", "Peru": "+51"}, "industries": ["Manufacturing", "Agriculture", "Energy"], "services": ["Operations", "Logistics"]},
    "AF": {"countries": ["S. Africa", "Mozambique", "Kenya", "Morocco"], "dial_code": {"S. Africa": "+27", "Mozambique": "+258", "Kenya": "+254", "Morocco": "+212"}, "industries": ["Telecommunications", "Mining", "Finance"], "services": ["Staffing", "Service"]}
}

# ------------------------------------------
# 2. Predefined Lead Data (for Company_Name, Country, Region, Lead_ID) - 108 Companies
# ------------------------------------------
predefined_leads = [
    # North America (20)
    ("Uber", "USA", "NA", "LDUBR-NA-001"), ("Walmart", "USA", "NA", "LDWMT-NA-002"), ("Ford Motor Co", "USA", "NA", "LDFRD-NA-003"), ("Amazon", "USA", "NA", "LDAMZ-NA-004"),
    ("Microsoft", "USA", "NA", "LDMST-NA-005"), ("Target", "USA", "NA", "LDTGT-NA-006"), ("Uber Eats", "USA", "NA", "LDUBE-NA-007"), ("CVS Health", "USA", "NA", "LDCVS-NA-008"),
    ("McKesson", "USA", "NA", "LDMCK-NA-009"), ("TD Bank Group", "Canada", "NA", "LDTDB-NA-010"), ("Shopify", "Canada", "NA", "LDSHF-NA-011"), ("Suncor Energy", "Canada", "NA", "LDSCR-NA-012"),
    ("Kaiser Permanente", "USA", "NA", "LDKPR-NA-013"), ("Wells Fargo", "USA", "NA", "LDWLF-NA-014"), ("Canadian Tire", "Canada", "NA", "LDCTC-NA-015"), ("American Tower", "USA", "NA", "LDAMT-NA-016"),
    ("Tim Hortons", "Canada", "NA", "LDTHC-NA-017"), ("Lowe's", "USA", "NA", "LDLWS-NA-018"), ("PicnicHealth", "USA", "NA", "LDPCH-NA-019"), ("Credit Saison India (USA operations)", "USA", "NA", "LDCSI-NA-020"),

    # Europe (16)
    ("Tesco", "UK", "EU", "LDTSCO-EU-021"), ("Lufthansa Group", "Germany", "EU", "LDLFT-EU-022"), ("Allianz Direct", "Germany", "EU", "LDALZ-EU-023"), ("HSBC", "UK", "EU", "LDHSBC-EU-024"),
    ("Sage Group", "UK", "EU", "LDSGE-EU-025"), ("Schneider Electric", "France", "EU", "LDSNE-EU-026"), ("Philips Healthcare", "Netherlands", "EU", "LDPHL-EU-027"), ("Vodafone", "UK", "EU", "LDVDF-EU-028"),
    ("Siemens", "Germany", "EU", "LDSMN-EU-029"), ("Nestlé", "Switzerland", "EU", "LDNST-EU-030"), ("ING Group", "Netherlands", "EU", "LDING-EU-031"), ("ABB", "Switzerland", "EU", "LDABB-EU-032"),
    ("Volvo Group", "Sweden", "EU", "LDVLV-EU-033"), ("Carrefour", "France", "EU", "LDCFR-EU-034"), ("ZF Group", "Germany", "EU", "LDZFG-EU-035"), ("Trigo", "UK", "EU", "LDTRG-EU-036"), # Assuming UK operations for Trigo

    # Asia-Pacific (14)
    ("DBS Bank", "Singapore", "APAC", "LDDBS-APAC-037"), ("Kiwibank", "New Zealand", "APAC", "LDKWB-APAC-038"), ("Reliance Retail", "India", "APAC", "LDRLR-APAC-039"), ("Tata Consultancy Services", "India", "APAC", "LDTCS-APAC-040"),
    ("SoftBank", "Japan", "APAC", "LDSFB-APAC-041"), ("Tokopedia / GoTo", "Indonesia", "APAC", "LDTKP-APAC-042"), ("Telstra", "Australia", "APAC", "LDTSL-APAC-043"), ("Westpac", "Australia", "APAC", "LDWSP-APAC-044"),
    ("infosys", "India", "APAC", "LDINF-APAC-045"), ("Grab", "Singapore", "APAC", "LDGRB-APAC-046"), ("JioMart", "India", "APAC", "LDJMR-APAC-047"), ("Toyota", "Japan", "APAC", "LDTYT-APAC-048"),
    ("Sony", "Japan", "APAC", "LDSNY-APAC-049"), ("Samsung", "S. Korea", "APAC", "LDSMS-APAC-050"),

    # Middle East (4)
    ("Saudi Aramco", "Saudi Arabia", "ME", "LDSAR-ME-051"), ("ADNOC", "UAE", "ME", "LDADC-ME-052"), ("QatarEnergy", "Qatar", "ME", "LDQTE-ME-053"), ("Investcorp", "Bahrain", "ME", "LDINV-ME-054"),

    # Latin America (10)
    ("Petrobras", "Brazil", "LA", "LDPBR-LA-055"), ("América Móvil", "Mexico", "LA", "LDAMX-LA-056"), ("MercadoLibre", "Argentina", "LA", "LDMCL-LA-057"), ("Codelco", "Chile", "LA", "LDCDL-LA-058"),
    ("Ecopetrol", "Colombia", "LA", "LDECP-LA-059"), ("Latam Airlines", "Chile", "LA", "LDLTM-LA-060"), ("Natura", "Brazil", "LA", "LDNTR-LA-061"), ("Itau Unibanco", "Brazil", "LA", "LDITU-LA-062"),
    ("Bancolombia", "Colombia", "LA", "LDBCL-LA-063"), ("Grupo Mexico", "Mexico", "LA", "LDGPM-LA-064"),

    # Africa (12)
    ("MTN Group", "S. Africa", "AF", "LDMTN-AF-065"), ("Sasol", "S. Africa", "AF", "LDSAL-AF-066"), ("Standard Bank Group", "S. Africa", "AF", "LDSBG-AF-067"), ("Vodacom", "S. Africa", "AF", "LDVDC-AF-068"),
    ("Safaricom", "Kenya", "AF", "LDSFC-AF-069"), ("KCB Group", "Kenya", "AF", "LDKCB-AF-070"), ("Attijariwafa Bank", "Morocco", "AF", "LDATJ-AF-071"), ("FirstRand", "S. Africa", "AF", "LDFRD-AF-072"),
    ("Absa Group", "S. Africa", "AF", "LDABG-AF-073"), ("Nedbank Group", "S. Africa", "AF", "LDNDB-AF-074"), ("Equity Group Holdings", "Kenya", "AF", "LDEGH-AF-075"), ("Telkom", "S. Africa", "AF", "LDTLK-AF-076"),

    # Adding more predefined companies to reach 108, maintaining the pattern and region assignment
    ("Infosys", "India", "APAC", "LDINF-APAC-077"), ("Wipro", "India", "APAC", "LDWIP-APAC-078"), ("HCL Technologies", "India", "APAC", "LDHCL-APAC-079"),
    ("Tech Mahindra", "India", "APAC", "LDTMH-APAC-080"), ("Larsen & Toubro Infotech", "India", "APAC", "LDLTI-APAC-081"), ("Mindtree", "India", "APAC", "LDMNT-APAC-082"),
    ("Mphasis", "India", "APAC", "LDMPH-APAC-083"), ("L&T Technology Services", "India", "APAC", "LDLTTS-APAC-084"), ("Persistent Systems", "India", "APAC", "LDPST-APAC-085"),
    ("Coforge", "India", "APAC", "LDCFG-APAC-086"), ("Zensar Technologies", "India", "APAC", "LDZNR-APAC-087"), ("Firstsource Solutions", "India", "APAC", "LDFRS-APAC-088"),
    ("ExlService Holdings", "India", "APAC", "LDEXL-APAC-089"), ("Genus Power Infrastructures", "India", "APAC", "LDGPI-APAC-090"), ("Cyient", "India", "APAC", "LDCYI-APAC-091"),

    ("Tata Motors", "India", "APAC", "LDTMO-APAC-092"), ("Mahindra & Mahindra", "India", "APAC", "LDMAM-APAC-093"), ("Maruti Suzuki India", "India", "APAC", "LDMSI-APAC-094"),
    ("Hyundai Motor India", "India", "APAC", "LDHMI-APAC-095"), ("Kia India", "India", "APAC", "LDKIA-APAC-096"), ("Toyota Kirloskar Motor", "India", "APAC", "LDTKM-APAC-097"),
    ("Honda Cars India", "India", "APAC", "LDHCI-APAC-098"), ("Renault India", "India", "APAC", "LDRNI-APAC-099"), ("Nissan Motor India", "India", "APAC", "LDNMI-APAC-100"),

    ("Apple", "USA", "NA", "LDAPL-NA-101"), ("Google", "USA", "NA", "LDGOO-NA-102"), ("Facebook", "USA", "NA", "LDFAB-NA-103"), ("Netflix", "USA", "NA", "LDNFL-NA-104"),
    ("Salesforce", "USA", "NA", "LDSFC-NA-105"), ("Tesla", "USA", "NA", "LDTES-NA-106"), ("IBM", "USA", "NA", "LDIBM-NA-107"), ("Intel", "USA", "NA", "LDINT-NA-108")
]


# ------------------------------------------
# 3. Generate Sales Reps (Unevenly distributed based on region size)
# ------------------------------------------
rep_names = [
    "Alice Smith", "Bob Johnson", "Charlie Lee", "Diana King", "Edward Brown",
    "Fiona Green", "George White", "Hannah Black", "Ian Gray", "Jill Lewis",
    "Kevin Martin", "Laura Scott", "Mike Turner", "Nina Adams", "Oscar Clark",
    "Paula Young", "Quinn Hall", "Rachel Allen", "Steve Wright", "Tina Baker"
] # 20 names

# Distribute 20 reps unevenly across regions (adjust numbers as needed to total 20)
# Example: NA: 4, EU: 4, APAC: 5, ME: 2, LA: 3, AF: 2
region_rep_distribution = {"NA": 4, "EU": 4, "APAC": 5, "ME": 2, "LA": 3, "AF": 2}

rep_list = []
rep_name_index = 0
for region, count in region_rep_distribution.items():
    for i in range(count):
        full_name = rep_names[rep_name_index]
        initials = "".join([part[0] for part in full_name.split()]).upper()
        rep_id = f"{initials}_{region}"
        rep_list.append({"rep_id": rep_id, "rep_name": full_name, "region": region})
        rep_name_index += 1

# Convert to DataFrame
rep_df = pd.DataFrame(rep_list)

# ------------------------------------------
# 4. Generate Leads DataFrame
# ------------------------------------------
def get_company_email(name):
    # Basic email domain generation
    domain = name.lower().replace(" ", "").replace("&", "").replace(".", "").replace("/", "").replace("(", "").replace(")", "") + ".com"
    return domain

# Use the generate_phone function defined in the previous cell (ad0fa588)

def generate_timestamp(start_days_ago=730): # Up to 2 years ago
    created = datetime.now() - timedelta(days=random.randint(0, start_days_ago), minutes=random.randint(0, 1440), seconds=random.randint(0, 86400))
    updated = created + timedelta(minutes=random.randint(10, 10000)) # Updated is after created
    return created.strftime("%Y-%m-%d %H:%M:%S"), updated.strftime("%Y-%m-%d %H:%M:%S")

def get_company_initials(name):
    return "".join([w[0] for w in name.split()]).upper()

# Define service categories and industries based on region_mapping for random lead generation
all_service_categories = list(set([service for region_data in region_mapping.values() for service in region_data.get('services', [])]))
all_industries = list(set([industry for region_data in region_mapping.values() for industry in region_data.get('industries', [])]))


# Generate a list of more descriptive company name parts for random leads
company_name_parts = {
    "adjectives": ["Global", "Strategic", "Innovative", "Dynamic", "Future", "Leading", "Creative", "Advanced"],
    "nouns": ["Solutions", "Systems", "Analytics", "Consulting", "Tech", "Ventures", "Partners", "Industries"]
}

def generate_random_company_name(adjectives, nouns):
    return f"{random.choice(adjectives)}{random.choice(nouns)}"

def get_abbreviated_company_name(name):
    # Simple abbreviation logic
    words = name.split()
    if len(words) > 1:
        return "".join([w[:2] for w in words]).upper() # Use first two letters of each word
    else:
        return name[:4].upper() # Use first four letters if single word


leads_data = []

# Add predefined leads
for company_name, country, region, lead_id in predefined_leads:
    company_email_domain = get_company_email(company_name)
    company_id = f"CMP_{get_company_initials(company_name)}"

    created_date, last_updated = generate_timestamp()
    deal_source = random.choices(["Inbound", "Outbound"], weights=[0.2, 0.8])[0]
    phone = generate_phone(country, region_mapping) # Use the new generate_phone function

    # Select a rep from same region
    region_reps = rep_df[rep_df["region"] == region]
    if not region_reps.empty:
        rep_row = region_reps.sample(n=1).iloc[0]
        assigned_rep_id = rep_row["rep_id"]
        rep_name = rep_row["rep_name"] # Keep rep_name for potential use
    else:
        # Handle case where no rep is available for the region
        assigned_rep_id = "Unassigned"
        rep_name = "Unassigned"

    lead_email = f"contact_{random.randint(1,100)}@{company_email_domain}"

    # Assign a service category and industry for predefined leads based on their country's typical services/industries
    # Find the region for the current country
    current_region = None
    for r, data in region_mapping.items():
        if country in data["countries"]:
            current_region = r
            break

    service_category = random.choice(region_mapping.get(current_region, {}).get('services', all_service_categories)) if current_region else random.choice(all_service_categories)
    company_industry = random.choice(region_mapping.get(current_region, {}).get('industries', all_industries)) if current_region else random.choice(all_industries)


    leads_data.append({
        "Lead_ID": lead_id,
        "Lead_Name": f"{company_name} Lead",
        "Company_Name": company_name,
        "Source": deal_source,
        "Region_ID": region,
        "Rep_ID": assigned_rep_id,
        "Created_Date": created_date,
        "Current_Stage": random.choice(["Lead Captured", "Qualified", "Contacted", "Demo Scheduled", "Proposal Sent", "Negotiation", "Deal Won", "Deal Lost"]),
        "Email": lead_email,
        "Phone": phone,
        "Priority": random.choice(["High", "Medium", "Low"]),
        "Last_Updated": last_updated,
        "Deal_Source": deal_source,
        "Company_ID": company_id,
        "Engagement_Level": random.choice(["High", "Medium", "Low"]),
        "Follow_Up_Threshold": random.choice([3, 5, 7, 10]),
        "Project_Term": random.choice(["0-6 months", "6-12 months", "12-24 months", ">24 months"]),
        "Budget_Tier": random.choice(["20-50L", "50-100L", "100L+"]),
        "Country": country,
        "Service_Category": service_category,
        "Company_Industry": company_industry
    })

# Generate additional random leads to reach 10,000
num_predefined_leads = len(predefined_leads)
num_additional_leads = 10000 - num_predefined_leads

# Get lists of all possible values for random generation
all_countries = [country for region_data in region_mapping.values() for country in region_data["countries"]]
all_regions = list(region_mapping.keys())

for i in range(num_additional_leads):
    company_name = generate_random_company_name(company_name_parts["adjectives"], company_name_parts["nouns"])
    country = random.choice(all_countries)
    # Find the region for the selected country using region_mapping
    region = next(r for r, data in region_mapping.items() if country in data["countries"])


    # Generate Lead ID in the desired format for random leads
    company_abbreviation = get_abbreviated_company_name(company_name)
    lead_id = f"LD{company_abbreviation}-{region}-{i+1:03}" # New format for random Lead IDs

    company_email_domain = get_company_email(company_name)
    company_id = f"CMP_{get_company_initials(company_name)}_{i+1:04}"

    created_date, last_updated = generate_timestamp()
    deal_source = random.choices(["Inbound", "Outbound"], weights=[0.2, 0.8])[0]
    phone = generate_phone(country, region_mapping) # Use the new generate_phone function

    # Select a rep from same region
    region_reps = rep_df[rep_df["region"] == region]
    if not region_reps.empty:
        rep_row = region_reps.sample(n=1).iloc[0]
        assigned_rep_id = rep_row["rep_id"]
        rep_name = rep_row["rep_name"]
    else:
        # Handle case where no rep is available for the region
        assigned_rep_id = "Unassigned"
        rep_name = "Unassigned"

    lead_email = f"contact_{random.randint(1,100)}@{company_email_domain}"

    # Assign a service category and industry for random leads based on the selected country's focus
    # Find the region for the current country
    current_region = None
    for r, data in region_mapping.items():
        if country in data["countries"]:
            current_region = r
            break

    service_category = random.choice(region_mapping.get(current_region, {}).get('services', all_service_categories)) if current_region else random.choice(all_service_categories)
    company_industry = random.choice(region_mapping.get(current_region, {}).get('industries', all_industries)) if current_region else random.choice(all_industries)


    # Generate other random data, potentially influenced by service_category if needed
    # (Currently, the random generation is independent of service_category,
    #  but can be made dependent here if specific correlations are desired)
    engagement_level = random.choice(["High", "Medium", "Low"])
    follow_up_threshold = random.choice([3, 5, 7, 10])
    project_term = random.choice(["0-6 months", "6-12 months", "12-24 months", ">24 months"])
    budget_tier = random.choice(["20-50L", "50-100L", "100L+"])
    current_stage = random.choice(["Lead Captured", "Qualified", "Contacted", "Demo Scheduled", "Proposal Sent", "Negotiation", "Deal Won", "Deal Lost"])
    priority = random.choice(["High", "Medium", "Low"])


    leads_data.append({
        "Lead_ID": lead_id, # Use the new formatted Lead ID
        "Lead_Name": f"{company_name} Lead",
        "Company_Name": company_name,
        "Source": deal_source, # Changed from "Lead Source" to "Source" to match requested column names
        "Region_ID": region,
        "Rep_ID": assigned_rep_id,
        "Created_Date": created_date,
        "Current_Stage": current_stage,
        "Email": lead_email,
        "Phone": phone,
        "Priority": priority,
        "Last_Updated": last_updated,
        "Deal_Source": deal_source,
        "Company_ID": company_id,
        "Engagement_Level": engagement_level,
        "Follow_Up_Threshold": follow_up_threshold,
        "Project_Term": project_term,
        "Budget_Tier": budget_tier,
        "Country": country,
        "Service_Category": service_category,
        "Company_Industry": company_industry
    })


df_lead = pd.DataFrame(leads_data)

# Reorder columns to match the requested order plus new columns
df_lead = df_lead[[
    "Lead_ID", "Lead_Name", "Company_Name", "Source", "Region_ID", "Rep_ID",
    "Created_Date", "Current_Stage", "Email", "Phone", "Priority", "Last_Updated",
    "Deal_Source", "Company_ID", "Engagement_Level", "Follow_Up_Threshold",
    "Project_Term", "Budget_Tier", "Country", "Company_Industry", "Service_Category"
]]


# ------------------------------------------
# 5. Export or View
# ------------------------------------------
# Save to CSV
# df_lead.to_csv("crm_leads_generated.csv", index=False)

# Display sample
display(df_lead.head())
display(df_lead.tail())
print(f"Generated {len(df_lead)} leads.")

,Lead_ID,Lead_Name,Company_Name,Source,Region_ID,Rep_ID,Created_Date,Current_Stage,Email,Phone,...,Last_Updated,Deal_Source,Company_ID,Engagement_Level,Follow_Up_Threshold,Project_Term,Budget_Tier,Country,Company_Industry,Service_Category
0,LDUBR-NA-001,Uber Lead,Uber,Outbound,NA,DK_NA,2023-10-10 11:51:06,Lead Captured,contact_65@uber.com,+1-3164569282,...,2023-10-12 06:31:06,Outbound,CMP_U,High,3,12-24 months,50-100L,USA,Finance,Data
1,LDWMT-NA-002,Walmart Lead,Walmart,Outbound,NA,CL_NA,2024-02-23 23:37:17,Contacted,contact_24@walmart.com,+1-3567801294,...,2024-02-27 11:14:17,Outbound,CMP_W,Medium,10,>24 months,50-100L,USA,Technology,Cloud
2,LDFRD-NA-003,Ford Motor Co Lead,Ford Motor Co,Outbound,NA,AS_NA,2025-05-01 09:53:57,Deal Won,contact_71@fordmotorco.com,+1-3558704844,...,2025-05-07 15:44:57,Outbound,CMP_FMC,Medium,3,6-12 months,20-50L,USA,Retail,Data
3,LDAMZ-NA-004,Amazon Lead,Amazon,Inbound,NA,DK_NA,2024-03-11 18:33:21,Deal Won,contact_42@amazon.com,+1-3670378066,...,2024-03-14 04:33:21,Inbound,CMP_A,Medium,3,6-12 months,50-100L,USA,Retail,Marketing
4,LDMST-NA-005,Microsoft Lead,Microsoft,Outbound,NA,AS_NA,2024-12-11 09:28:36,Negotiation,contact_94@microsoft.com,+1-3586323610,...,2024-12-11 11:33:36,Outbound,CMP_M,High,7,0-6 months,20-50L,USA,Retail,Cloud


,Lead_ID,Lead_Name,Company_Name,Source,Region_ID,Rep_ID,Created_Date,Current_Stage,Email,Phone,...,Last_Updated,Deal_Source,Company_ID,Engagement_Level,Follow_Up_Threshold,Project_Term,Budget_Tier,Country,Company_Industry,Service_Category
9995,LDLEAD-APAC-9888,LeadingConsulting Lead,LeadingConsulting,Outbound,APAC,LS_APAC,2024-06-19 11:39:34,Contacted,contact_23@leadingconsulting.com,+64-2598061890,...,2024-06-19 14:18:34,Outbound,CMP_L_9888,High,7,12-24 months,100L+,New Zealand,E-Commerce,Staffing
9996,LDSTRA-EU-9889,StrategicPartners Lead,StrategicPartners,Inbound,EU,EB_EU,2024-05-30 02:45:48,Contacted,contact_85@strategicpartners.com,+31-7401780756,...,2024-05-31 05:11:48,Inbound,CMP_S_9889,High,3,12-24 months,50-100L,Netherlands,Pharmaceuticals,Engineering
9997,LDCREA-LA-9890,CreativeSolutions Lead,CreativeSolutions,Outbound,LA,RA_LA,2024-02-06 04:39:56,Deal Won,contact_84@creativesolutions.com,+57-8540010960,...,2024-02-08 09:24:56,Outbound,CMP_C_9890,High,10,>24 months,50-100L,Colombia,Manufacturing,Operations
9998,LDINNO-LA-9891,InnovativeSolutions Lead,InnovativeSolutions,Outbound,LA,PY_LA,2025-04-07 04:18:05,Proposal Sent,contact_87@innovativesolutions.com,+57-6391146298,...,2025-04-11 17:38:05,Outbound,CMP_I_9891,High,7,6-12 months,20-50L,Colombia,Agriculture,Logistics
9999,LDCREA-APAC-9892,CreativeSystems Lead,CreativeSystems,Inbound,APAC,KM_APAC,2024-06-21 05:39:10,Proposal Sent,contact_56@creativesystems.com,+86-5350282413,...,2024-06-26 12:50:10,Inbound,CMP_C_9892,Low,3,12-24 months,50-100L,China,E-Commerce,AI


Generated 10000 leads.


In [18]:
#REGION
import pandas as pd
import numpy as np
import random

# Use unique Region_IDs from df_lead
region_ids = df_lead['Region_ID'].unique().tolist()
timezones = ["IST", "EST", "PST", "CET", "GMT"]
countries_in_leads = df_lead['Country'].unique().tolist() # Get countries from df_lead

region_data = []

for region_id in region_ids:
    # Select a country that belongs to this region based on region_mapping
    country = random.choice([c for r, data in region_mapping.items() if r == region_id for c in data['countries'] if c in countries_in_leads])

    # Find reps assigned to this region
    reps_in_region = rep_df[rep_df['region'] == region_id]['rep_name'].tolist()
    region_head = random.choice(reps_in_region) if reps_in_region else "Unassigned"

    region_data.append({
        "Region_ID": region_id,
        "Region_Name": f"Region_{region_id}",
        "Region_Head": region_head,
        "Timezone": random.choice(timezones),
        "Region_Target": round(random.uniform(500000, 5000000), 2)
    })

df_region = pd.DataFrame(region_data)
display(df_region.head())

,Region_ID,Region_Name,Region_Head,Timezone,Region_Target
0,NA,Region_NA,Alice Smith,PST,3224572.33
1,EU,Region_EU,Edward Brown,CET,2399360.05
2,APAC,Region_APAC,Ian Gray,PST,3191755.76
3,ME,Region_ME,Nina Adams,CET,940483.69
4,LA,Region_LA,Rachel Allen,GMT,1485508.44


In [8]:
#SALES_REP
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Use unique Rep_IDs and their corresponding regions from df_lead
sales_rep_data = []
added_rep_ids = set()

for index, row in df_lead.iterrows():
    rep_id = row['Rep_ID']
    region_id = row['Region_ID']
    # Add rep data only if not already added
    if rep_id not in added_rep_ids:
        # Find the rep name from the rep_df based on Rep_ID
        rep_info = rep_df[rep_df['rep_id'] == rep_id]
        rep_name = rep_info['rep_name'].iloc[0] if not rep_info.empty else f"Rep_{rep_id}"


        sales_rep_data.append({
            "Rep_ID": rep_id,
            "Rep_Name": rep_name, # Use name from rep_df
            "Email": f"{rep_id.lower()}@crmcorp.com", # Generate email based on Rep_ID
            "Phone": f"+91-9{random.randint(100000000,999999999)}", # Keep existing phone generation for now
            "Region_ID": region_id,
            "Join_Date": datetime(2020, 1, 1) + timedelta(days=random.randint(0, 1000)),
            "Current_Status": random.choice(["Active", "Inactive", "On Leave"]),
            "Performance_Rating": round(random.uniform(1.0, 5.0), 1),
            "Total_Deals_Closed": random.randint(0, 500),
            "Avg_conversion_time": round(random.uniform(3.0, 30.0), 2),
            "Win_rate": round(random.uniform(0.1, 0.9), 2)
        })
        added_rep_ids.add(rep_id)

# Add any reps from rep_df who are not assigned to any lead in df_lead
reps_not_in_leads = rep_df[~rep_df['rep_id'].isin(added_rep_ids)]
for index, rep_row in reps_not_in_leads.iterrows():
    rep_id = rep_row['rep_id']
    region_id = rep_row['region']
    rep_name = rep_row['rep_name']

    sales_rep_data.append({
        "Rep_ID": rep_id,
        "Rep_Name": rep_name,
        "Email": f"{rep_id.lower()}@crmcorp.com",
        "Phone": f"+91-9{random.randint(100000000,999999999)}",
        "Region_ID": region_id,
        "Join_Date": datetime(2020, 1, 1) + timedelta(days=random.randint(0, 1000)),
        "Current_Status": random.choice(["Active", "Inactive", "On Leave"]),
        "Performance_Rating": round(random.uniform(1.0, 5.0), 1),
        "Total_Deals_Closed": random.randint(0, 500),
        "Avg_conversion_time": round(random.uniform(3.0, 30.0), 2),
        "Win_rate": round(random.uniform(0.1, 0.9), 2)
    })


df_sales_rep = pd.DataFrame(sales_rep_data)
display(df_sales_rep.head())
display(df_sales_rep.tail())
print(f"Generated {len(df_sales_rep)} sales reps.")

,Rep_ID,Rep_Name,Email,Phone,Region_ID,Join_Date,Current_Status,Performance_Rating,Total_Deals_Closed,Avg_conversion_time,Win_rate
0,DK_NA,Diana King,dk_na@crmcorp.com,+91-9154606603,NA,2021-05-20,On Leave,4.6,193,25.06,0.67
1,CL_NA,Charlie Lee,cl_na@crmcorp.com,+91-9875155404,NA,2020-02-09,On Leave,2.5,22,8.46,0.73
2,AS_NA,Alice Smith,as_na@crmcorp.com,+91-9802248343,NA,2021-10-06,Active,2.3,327,13.59,0.70
3,BJ_NA,Bob Johnson,bj_na@crmcorp.com,+91-9581704652,NA,2020-11-30,On Leave,4.1,363,17.39,0.59
4,GW_EU,George White,gw_eu@crmcorp.com,+91-9712490751,EU,2020-07-10,On Leave,2.7,250,19.21,0.42


,Rep_ID,Rep_Name,Email,Phone,Region_ID,Join_Date,Current_Status,Performance_Rating,Total_Deals_Closed,Avg_conversion_time,Win_rate
15,QH_LA,Quinn Hall,qh_la@crmcorp.com,+91-9994975441,LA,2021-08-17,On Leave,1.2,82,8.14,0.43
16,PY_LA,Paula Young,py_la@crmcorp.com,+91-9953082073,LA,2022-02-15,On Leave,4.3,406,3.90,0.88
17,SW_AF,Steve Wright,sw_af@crmcorp.com,+91-9490732991,AF,2021-05-25,Active,2.1,109,12.30,0.67
18,TB_AF,Tina Baker,tb_af@crmcorp.com,+91-9795114898,AF,2022-09-26,On Leave,2.6,383,23.28,0.38
19,RA_LA,Rachel Allen,ra_la@crmcorp.com,+91-9965905822,LA,2021-08-22,Active,4.7,196,20.36,0.40


Generated 20 sales reps.


In [9]:
#COMPANIES
import pandas as pd
import numpy as np
import random

industries = ["Technology", "Healthcare", "Finance", "Manufacturing", "Retail", "E-Commerce", "Green Tech", "Mobility", "Real Estate", "Banking", "Logistics", "Housing", "Energy"] # Expanded industries
sizes = ["Start-up", "Small business", "Medium Enterprise", "Large Enterprise"]
ownerships = ["Public Sector", "Private Sector", "Joint Venture", "Franchise"]
funding = ["Bootstrapped", "Seed", "Series A", "Series B", "IPO"]
interest_areas = ["Data", "Marketing", "Cloud", "Design", "HR", "AI", "Consulting", "Engineering", "Operations", "Staffing", "T&M"] # Expanded interest areas

company_data = []
added_company_ids = set()

# Add companies from df_lead
for index, row in df_lead.iterrows():
    company_id = row['Company_ID']
    company_name = row['Company_Name']
    country = row['Country']
    # Add company data only if not already added
    if company_id not in added_company_ids:
        # Attempt to get industry and service from region_mapping if available, otherwise use random
        # Find the country info in the region_mapping dictionary
        current_region_data = next((data for region, data in region_mapping.items() if country in data["countries"]), None)

        company_industry = random.choice(current_region_data.get('industries', industries)) if current_region_data and current_region_data.get('industries') else random.choice(industries)
        interest_area = random.choice(current_region_data.get('services', interest_areas)) if current_region_data and current_region_data.get('services') else random.choice(interest_areas)

        company_data.append({
            "Company_ID": company_id,
            "Company_Name": company_name,
            "Company_Industry": company_industry, # Use industry from region_mapping or random
            "Company_Size": random.choice(sizes),
            "Company_Ownership": random.choice(ownerships),
            "Company_Funding": random.choice(funding),
            "Country": country, # Use country from df_lead
            "Interest_Area": interest_area # Use service from region_mapping or random
        })
        added_company_ids.add(company_id)

# Generate additional companies if needed (optional, based on whether df_lead has all companies)
# If df_lead contains all the companies you need for df_company, you might not need this part.
# Assuming df_lead has the comprehensive list of companies you want in df_company.
# (No additional generation needed as df_lead now contains 10000 companies)

df_company = pd.DataFrame(company_data)
display(df_company.head())
display(df_company.tail())
print(f"Generated {len(df_company)} companies.")

,Company_ID,Company_Name,Company_Industry,Company_Size,Company_Ownership,Company_Funding,Country,Interest_Area
0,CMP_U,Uber,Healthcare,Large Enterprise,Public Sector,Series A,USA,Marketing
1,CMP_W,Walmart,Finance,Medium Enterprise,Public Sector,Seed,USA,Data
2,CMP_FMC,Ford Motor Co,Technology,Medium Enterprise,Joint Venture,IPO,USA,Marketing
3,CMP_A,Amazon,Retail,Large Enterprise,Public Sector,Seed,USA,Cloud
4,CMP_M,Microsoft,Healthcare,Small business,Private Sector,Bootstrapped,USA,Marketing


,Company_ID,Company_Name,Company_Industry,Company_Size,Company_Ownership,Company_Funding,Country,Interest_Area
9958,CMP_L_9888,LeadingConsulting,Technology,Small business,Public Sector,Series B,New Zealand,AI
9959,CMP_S_9889,StrategicPartners,Banking,Start-up,Franchise,Seed,Netherlands,Operations
9960,CMP_C_9890,CreativeSolutions,Agriculture,Small business,Private Sector,Seed,Colombia,Logistics
9961,CMP_I_9891,InnovativeSolutions,Agriculture,Small business,Joint Venture,Series A,Colombia,Operations
9962,CMP_C_9892,CreativeSystems,E-Commerce,Medium Enterprise,Public Sector,Series A,China,Staffing


Generated 9963 companies.


In [16]:
#STAGE HISTORY
import pandas as pd
import random
from datetime import datetime, timedelta

stage_history_data = []
history_id = 1
stages = ["Lead Captured", "Qualified", "Contacted", "Demo Scheduled", "Proposal Sent", "Negotiation", "Deal Won", "Deal Lost"] # Define stages

for index, lead in df_lead.iterrows():
    num_stages = random.randint(1, len(stages)) # A lead can go through a random number of stages
    # Ensure the final stage in the history matches the Current_Stage in df_lead
    lead_stages = random.sample(stages, num_stages)
    if lead['Current_Stage'] not in lead_stages:
        # Replace a random stage with the current stage if it's not present
        lead_stages[random.randint(0, num_stages - 1)] = lead['Current_Stage']
    # Ensure the current stage is the last one
    if lead_stages[-1] != lead['Current_Stage']:
         lead_stages.remove(lead['Current_Stage'])
         lead_stages.append(lead['Current_Stage'])

    start_date = datetime.strptime(lead.Created_Date, "%Y-%m-%d %H:%M:%S")
    # Sort stages to ensure chronological order (optional, but makes more sense)
    # For simplicity, we'll just assign timestamps sequentially

    # Filter stages to ensure chronological order based on the defined stages list
    ordered_stages = [stage for stage in stages if stage in lead_stages]
    # Ensure the last stage is the current stage from df_lead
    if ordered_stages and ordered_stages[-1] != lead['Current_Stage']:
         if lead['Current_Stage'] in ordered_stages:
              ordered_stages.remove(lead['Current_Stage'])
         ordered_stages.append(lead['Current_Stage'])
    elif not ordered_stages and lead['Current_Stage']:
         ordered_stages = [lead['Current_Stage']]


    current_timestamp = start_date
    for seq, stage in enumerate(ordered_stages, start=1):
        # Ensure timestamp is after the previous stage's timestamp and before Last_Updated
        if seq == len(ordered_stages):
             # The last stage timestamp should be the Last_Updated time
             timestamp = datetime.strptime(lead.Last_Updated, "%Y-%m-%d %H:%M:%S")
        else:
             # Generate a random timestamp between the current timestamp and the lead's Last_Updated time
             # distributed across the remaining stages
             remaining_stages = len(ordered_stages) - seq
             time_until_last_update = (datetime.strptime(lead.Last_Updated, "%Y-%m-%d %H:%M:%S") - current_timestamp).total_seconds()
             if remaining_stages > 0:
                 # Distribute the remaining time among the remaining stages
                 time_increment = time_until_last_update / remaining_stages
                 timestamp = current_timestamp + timedelta(seconds=random.randint(1, max(int(time_increment), 60))) # Ensure at least 1 minute difference
             else:
                 # Should not happen if the last stage is handled correctly, but as a fallback
                 timestamp = datetime.strptime(lead.Last_Updated, "%Y-%m-%d %H:%M:%S")

        # Set Is_Rentry to False if the lead's Current_Stage is "Deal Won"
        is_rentry = False if lead['Current_Stage'] == 'Deal Won' else random.choice([True, False])

        stage_history_data.append({
            "History_ID": history_id,
            "Lead_ID": lead.Lead_ID,
            "Stage": stage,
            "Timestamp": timestamp.strftime("%Y-%m-%d %H:%M:%S"), # Format timestamp
            "Rep_ID": lead.Rep_ID,
            "Stage_Sequence": seq,
            "Is_Rentry": is_rentry, # Use the determined is_rentry value
            "Notes": f"Notes for stage {stage} for Lead {lead.Lead_ID}"
        })
        history_id += 1
        current_timestamp = timestamp # Update current_timestamp for the next iteration


df_stage_history = pd.DataFrame(stage_history_data)
display(df_stage_history.head())
display(df_stage_history.tail())
print(f"Generated {len(df_stage_history)} stage history entries.")

,History_ID,Lead_ID,Stage,Timestamp,Rep_ID,Stage_Sequence,Is_Rentry,Notes
0,1,LDUBR-NA-001,Contacted,2023-10-10 17:24:10,DK_NA,1,True,Notes for stage Contacted for Lead LDUBR-NA-001
1,2,LDUBR-NA-001,Negotiation,2023-10-12 06:04:38,DK_NA,2,False,Notes for stage Negotiation for Lead LDUBR-NA-001
2,3,LDUBR-NA-001,Lead Captured,2023-10-12 06:31:06,DK_NA,3,False,Notes for stage Lead Captured for Lead LDUBR-N...
3,4,LDWMT-NA-002,Qualified,2024-02-24 07:40:43,CL_NA,1,False,Notes for stage Qualified for Lead LDWMT-NA-002
4,5,LDWMT-NA-002,Demo Scheduled,2024-02-25 07:41:16,CL_NA,2,True,Notes for stage Demo Scheduled for Lead LDWMT-...


,History_ID,Lead_ID,Stage,Timestamp,Rep_ID,Stage_Sequence,Is_Rentry,Notes
44687,44688,LDINNO-LA-9891,Lead Captured,2025-04-07 15:44:48,PY_LA,1,False,Notes for stage Lead Captured for Lead LDINNO-...
44688,44689,LDINNO-LA-9891,Demo Scheduled,2025-04-09 14:24:31,PY_LA,2,True,Notes for stage Demo Scheduled for Lead LDINNO...
44689,44690,LDINNO-LA-9891,Proposal Sent,2025-04-11 17:38:05,PY_LA,3,True,Notes for stage Proposal Sent for Lead LDINNO-...
44690,44691,LDCREA-APAC-9892,Lead Captured,2024-06-22 15:44:46,KM_APAC,1,False,Notes for stage Lead Captured for Lead LDCREA-...
44691,44692,LDCREA-APAC-9892,Proposal Sent,2024-06-26 12:50:10,KM_APAC,2,False,Notes for stage Proposal Sent for Lead LDCREA-...


Generated 44692 stage history entries.


In [11]:
!pip install --upgrade gspread gspread_dataframe

import gspread
from gspread_dataframe import set_with_dataframe
from google.colab import auth
auth.authenticate_user()


In [13]:
# Use the spreadsheet name you created
spreadsheet = client.open("CRM_Capstone")

In [ ]:
# Sheet mappings
sheet_mapping = {
    "Leads": df_lead,
    "Sales_Reps": df_sales_rep,
    "Regions": df_region,
    "Stage_History": df_stage_history,
    "Companies": df_company
}

# Write to each sheet
for sheet_name, df in sheet_mapping.items():
    worksheet = spreadsheet.worksheet(sheet_name)
    worksheet.clear()
    set_with_dataframe(worksheet, df)
